## Computational Chemistry for Experimentalists
## Module 10: Transition States 

Transition state theory is central to our understanding of reaction kinetics. Here we compute transition states and use them to approximate reaction rates.

This first block imports all of the necessary Python modules. If these aren't installed, this will fail. 

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from pyscf import gto,scf
from pyscf.tools import cubegen 
import py3Dmol
import numpy 
import matplotlib.pyplot as plt
from pyscf.geomopt.geometric_solver import optimize
from rdkit.Geometry import Point3D
from rdkit.Chem import rdDetermineBonds
from scipy.integrate import odeint 

## Part 1: Thioformic acid tautomerization transition state 

Predict the barrier to proton transfer in HO-C(H)=S to O=C(H)-SH . First we compute optimized reactant and product geometries, starting from reasonable guesses generated in Gaussian 

In [ ]:
basis='3-21g'
r='''C 0.456789    0.501770   -0.000119;
O   1.575978   -0.208819    0.000003;
H  0.674888    1.549751    0.000122;
H 1.425887   -1.168284    0.1;
 S   -1.090583   -0.107596    0.000035;
'''
p=''' C   -0.723601    0.447486    0.000004;
 O   -1.612622   -0.347749   -0.000001;
 H    -0.829152    1.520621   -0.000008;
 H   0.890735   -1.343410    0.1;
 S   1.073812   -0.005008   -0.000001;
'''
ts='''C    0.637965    0.613800    0.000021;
O      1.688795   -0.062134   -0.000007;
H    0.628619    1.684246    0.000047;
H     0.830742   -1.165795   -0.000023;
 S   -0.743162   -0.503296    0.1;
'''

In [ ]:
%%capture cap 
mr=gto.Mole(atom=r,basis=basis)
mr.build()
mfr=scf.UHF(mr)
mfr.kernel()
mor= mfr.Gradients().optimizer(solver='geomeTRIC').kernel()

In [ ]:
with open('TFAR.txt', 'w') as file:
    file.write(cap.stdout)

In [ ]:
%%capture cap 
mp=gto.Mole(atom=p,basis=basis)
mp.build()
mfp=scf.UHF(mp)
mfp.kernel()
mop= mfp.Gradients().optimizer(solver='geomeTRIC').kernel()

In [ ]:
with open('TFAP.txt', 'w') as file:
    file.write(cap.stdout)

In [ ]:
%%capture cap 
mt=gto.Mole(atom=ts,basis=basis)
mt.build()
mft=scf.UHF(mt)
mft.kernel()
params={'transition':True}
mot=mft.Gradients().optimizer(solver='geomeTRIC').kernel(params)


In [ ]:
with open('TFATS.txt', 'w') as file:
    file.write(cap.stdout)

Generate new SCF calculations at the optimized reactant, transition state, and product geometries, and compute the forward reaction barrier and reaction energy in kJ/mol 

In [ ]:
mfro=scf.UHF(mor)
mfro.kernel()
mfpo=scf.UHF(mop)
mfpo.kernel()
mfto=scf.UHF(mot)
mfto.kernel()
DE=2625.5*(mfpo.e_tot-mfro.e_tot)
FB=2625.5*(mfto.e_tot-mfro.e_tot)
print('Reaction energy: %.2f kJ/mol'%(DE))
print('Forward barrier: %.2f kJ/mol'%(FB))

Use  MolFromXYZFile to generate RDKit molecules and Mol blocks to visualize the computed reactant, transition state, and product. The computed bond orders won't be reasonable, but we can at least see the geometry. 

In [ ]:
ms=[mor,mot,mop]
p=py3Dmol.view(width=600,height=200,viewergrid=(1,3))

msr=[]
for i in range(3):
    m=ms[i]
    m.tofile('t.xyz')
    raw_m=Chem.MolFromXYZFile('t.xyz')
    m2=Chem.Mol(raw_m)
    rdDetermineBonds.DetermineBonds(m2)
    msr.append(m2)
    mb=Chem.MolToMolBlock(m2)
    #print(mb)
    p.addModel(mb,'sdf',viewer=(0,i))
p.setStyle({'stick':{},'sphere':{"scale":0.3}})
p.zoomTo()
p.show()

## Part 2: Energy Level Diagrams: Water-Catalyzed thioformic acid tautomerization 

We first write out all species present in all intermediates and transition states 


HO-CH=S + H2O --> HO-CH=S..HOH --> TS --> HOH..O=CH-SH --> O=CH-SH + H2O


We then compute the optimized geometries and energies of the new intermediates and transition state , as well as the free wter catalyst 

In [ ]:
w='O 0.0 0.0 0.0; H 0.0 0.0 1.2; H 0.0 1.0 -0.2 '
rc=''' C   0.758714    0.888018    0.014296;
 O   -0.444359    1.371224   -0.010302;
  H   1.508356    1.654087    0.044841;
  H  -1.190598    0.685250   -0.041201;
 S     1.207301   -0.744965    0.005166;
 O    -2.056292   -0.576654   -0.062245;
  H    -2.746439   -0.790579    0.576670;
 H     -1.435207   -1.313986   -0.168370;
'''
pc='''   C   0.868704    0.923096    0.016671;
 O    -0.162555    1.549226   -0.004200;
  H  1.845212    1.380007    0.047723;
 H     -1.765877    0.458766   -0.117259;
 S     0.982041   -0.900511    0.002083;
  O  -2.077177   -0.461847   -0.078785;
   H    -2.708668   -0.562710    0.644696;
 H    -0.377693   -1.105494   -0.044639;
'''
tsc=''' C -0.817401   -1.400810    0.391573;
  O  -1.303642   -0.792867   -0.529742;
 H    -1.182824   -2.355461    0.736590;
  H   -0.440290    0.899595   -0.931101;
  S   0.614435   -0.847162    1.381950;
  O   0.294822    1.450928   -0.618932;
  H     0.375281    2.260526   -1.135455;
 H   0.807874    0.330930    0.704880;
'''

In [ ]:
mw=gto.Mole(atom=w,basis=basis)
mw.build()
mfw=scf.UHF(mw)
mfw.kernel()
mow= mfw.Gradients().optimizer(solver='geomeTRIC').kernel()
mrc=gto.Mole(atom=rc,basis=basis)
mrc.build()
mfrc=scf.UHF(mrc)
mfrc.kernel()
morc= mfrc.Gradients().optimizer(solver='geomeTRIC').kernel()
mpc=gto.Mole(atom=pc,basis=basis)
mpc.build()
mfpc=scf.UHF(mpc)
mfpc.kernel()
mopc= mfpc.Gradients().optimizer(solver='geomeTRIC').kernel()

In [ ]:
mtc=gto.Mole(atom=tsc,basis=basis)
mtc.build()
mftc=scf.UHF(mtc)
mftc.kernel()
params={'transition':True}
motc= mftc.Gradients().optimizer(solver='geomeTRIC').kernel(params)

Visualize the complex geometries 

In [ ]:
ms=[morc,motc,mopc]
p=py3Dmol.view(width=600,height=200,viewergrid=(1,3))
msrc=[]
for i in range(3):
    m=ms[i]
    m.tofile('t.xyz')
    raw_m=Chem.MolFromXYZFile('t.xyz')
    m2=Chem.Mol(raw_m)
    rdDetermineBonds.DetermineBonds(m2)
    msrc.append(m2)
    mb=Chem.MolToMolBlock(m2)
    #print(mb)
    p.addModel(mb,'sdf',viewer=(0,i))
p.setStyle({'stick':{},'sphere':{"scale":0.3}})
p.zoomTo()
p.show()

Compute the relative energies of all species, noting that the free reactant and free product must include the energy of free H2O . The overall reaction enery is still 34 kJ/mol, however the barrier is much lower. At this level of theory, the transition state is lower in energy than the free reactants. 

In [ ]:
mfwo=scf.UHF(mow)
mfwo.kernel()
mfrco=scf.UHF(morc)
mfrco.kernel()
mftco=scf.UHF(motc)
mftco.kernel()
mfpco=scf.UHF(mopc)
mfpco.kernel()
Eref=mfro.e_tot+mfwo.e_tot
DEs=2625.5*numpy.array([Eref-Eref,mfrco.e_tot-Eref,mftco.e_tot-Eref,mfpco.e_tot-Eref,mfpo.e_tot+mfwo.e_tot-Eref])
print(DEs)

## Part 3: Reaction Dynamics Simulations 
We use SciPy numerical integration to compute the partial pressures of H2O, reactant, RC, TC,PC, and free product  as a function of time. To do this we need to set up the chemical kinetics equations for all five species and choose initial conditions. Our five species undergo six reactions 

H2O + R --> RC (k0)

RC--> H2O + R  (k0 Exp[-(G(R)+G(H2O)-G(RC)

RC--> PC       (k1 



PC -->  H2O + P

P + H2O ---> PC

We assume that the barrier to the complexation reactions is 10 kJ/mol, that k0 is the Arrhenius term in the rate constants. 



In [ ]:
RT=(0.008314*1000)
k0=100 # Arrhenius constant, 1/seconds  
CT=10 # Barrier to complexation , kJ/mo 

# This step creates the rate constants for all six reactions 
[kf1,kb1,kf2,kb2,kf3,kb3]=[k0*numpy.exp(-CT/RT),
                           k0*numpy.exp(-((DEs[0]-DEs[1])+CT)/RT),
                           k0*numpy.exp(-(DEs[2]-DEs[1])/RT),
                           k0*numpy.exp(-(DEs[2]-DEs[3])/RT),
                           k0*numpy.exp(-((DEs[4]-DEs[3])+CT)/RT),
                           k0*numpy.exp(-CT/RT)]
print([kf1,kb1,kf2,kb2,kf3,kb3])
def reaction(y,t):
    r,rc,pc,p,w = y # Concentrations of all five species 
    rs=[kf1*r*w,kb1*rc,kf2*rc,kb2*pc,kf3*pc,kb3*p*w] # rates are rate constants times concentrations 
    drdt=-rs[0]+rs[1] # Reactant is consumed in the first forward reaction, produced in first backward
    drcdt=rs[0]-rs[1]-rs[2]+rs[3] # Complex is involved in reactions 1-4 
    dpcdt=rs[2]-rs[3]-rs[4]+rs[5]
    dpdt=rs[4]-rs[5]
    dwdt=-rs[0]+rs[1]+rs[4]-rs[5]
    return [drdt,drcdt,dpcdt,dpdt,dwdt] # Time derivatives of all concentrations 

In [ ]:
y0=[0,0,0,1,0.1] # Initial concentrations
t=numpy.linspace(start=10**(-3), stop=10**6, num=10**7) # Time points 

solution=odeint(reaction,y0,t)
plt.plot(t, solution[:,3], label='Free HSC(=O)H reactant ')
plt.plot(t, solution[:, 1], label='Complexed HOC(=S)H')
plt.plot(t, solution[:, 2], label='Complexed HSC(=O)H')
plt.plot(t, solution[:, 0], label='Free HOC(=S)H product',color='Black')
plt.plot(t, solution[:, 4], label='Free water',linestyle='--',color='Blue')

plt.semilogx()
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.legend()
plt.show()

Your homework is to determine the concentration as a function of time for autocatalytic decomposition of HSC(=O)H in the gas phase. In autocatalytic decomposition, two molecules of HSC(=O)H complex with each other and react to form two molecules of HOC(=S)H. TRy to generate a plot like the one above. 